# Computed Formant Error 
computed relative formant error of each model
- Created 1 April 2020

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import librosa
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import librosa.display
from matplotlib import cm
from functools import partial

import sklearn
from sklearn.decomposition import PCA
from os import makedirs
import os
from os.path import join
import shutil
from IPython import display

%matplotlib inline

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# np.random.seed(42)

In [3]:
def numpy_fillna(data):
    lens = np.array([len(i) for i in data])
    mask = np.arange(lens.max()) < lens[:,None]
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out

### Monosyllablic vowel utterance 

In [12]:
EXP = [120,121,122,123,141]

In [16]:
res = []
for exp_num in EXP:
    path = '../experiment_2/result/eval_{}/formant/'.format(exp_num)

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))
    a3 = np.load(join(path, 'actual_F3.npy'))

    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))
    e3 = np.load(join(path, 'estimated_F3.npy'))

    f1_error = np.absolute(np.mean(a1 - e1))
    f2_error = np.absolute(np.mean(a2 - e2))
    f3_error = np.absolute(np.mean(a3 - e3))
    res.append([exp_num, f1_error, f2_error, f3_error])
    
pd.DataFrame(np.array(res),columns=['MODEL', 'F1 ERROR','F2 ERROR', 'F3 ERROR'])

,MODEL,F1 ERROR,F2 ERROR,F3 ERROR
0,120.0,29.459646,49.209462,84.091433
1,121.0,9.320058,59.699235,111.625765
2,122.0,0.260055,19.167871,19.545151
3,123.0,9.899702,4.953315,23.007406
4,141.0,6.484798,1.032890,2.868631


### Disyllablic vowel utterance 

In [5]:
EXP = [125,126,127,128,140]

In [26]:
res = []
for exp_num in EXP:

    path = '../experiment_2/result/eval_{}/formant/'.format(exp_num)

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))
    a3 = np.load(join(path, 'actual_F3.npy'))
    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))
    e3 = np.load(join(path, 'estimated_F3.npy'))
    
    
    
    e1 = numpy_fillna(e1)
    e2 = numpy_fillna(e2)
    e3 = numpy_fillna(e3)
    

    f1_error = a1 - e1
    f2_error = a2 - e2
    f3_error = a3 - e3

    
    f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
    f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
    f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))
    
    f11_error = np.absolute(np.mean(f1_error[:,0,:]))
    f21_error = np.absolute(np.mean(f1_error[:,1,:]))
    f12_error = np.absolute(np.mean(f2_error[:,0,:]))
    f22_error = np.absolute(np.mean(f2_error[:,1,:]))
    f13_error = np.absolute(np.mean(f3_error[:,0,:]))
    f23_error = np.absolute(np.mean(f3_error[:,1,:]))

    res.append([exp_num, f11_error, f12_error, f13_error, f21_error, f22_error, f23_error])

pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR', '2F1 ERROR','2F2 ERROR', '2F3 ERROR'])

,MODEL,1F1 ERROR,1F2 ERROR,1F3 ERROR,2F1 ERROR,2F2 ERROR,2F3 ERROR
0,125.0,153.381740,275.305721,199.263454,93.942678,176.689423,98.764477
1,126.0,19.697726,100.688060,176.969772,17.613453,96.309370,161.606790
2,127.0,5.554327,14.265457,32.530347,5.303880,17.824886,41.797299
3,128.0,6.911143,3.313094,4.252541,3.244223,7.685816,12.422432
4,140.0,8.083394,1.074850,23.892439,4.224894,1.318200,23.449496


## Formant Error of each syllable 

### Monosyllablic vowel utterance 

In [34]:
exp_num = 141

In [35]:
path = '../experiment_2/result/eval_{}/formant/'.format(exp_num)

a1 = np.load(join(path, 'actual_F1.npy'))
a2 = np.load(join(path, 'actual_F2.npy'))
a3 = np.load(join(path, 'actual_F3.npy'))

e1 = np.load(join(path, 'estimated_F1.npy'))
e2 = np.load(join(path, 'estimated_F2.npy'))
e3 = np.load(join(path, 'estimated_F3.npy'))

f1_error = np.absolute(np.mean(a1 - e1, axis=1))
f2_error = np.absolute(np.mean(a2 - e2, axis=1))
f3_error = np.absolute(np.mean(a3 - e3, axis=1))


pd.DataFrame({ 'F1 ERROR': f1_error,'F2 ERROR':f2_error, 'F3 ERROR':f3_error},columns=['F1 ERROR','F2 ERROR', 'F3 ERROR'])

,F1 ERROR,F2 ERROR,F3 ERROR
0,9.193027,1.151415,61.032071
1,38.323501,23.791440,138.983825
2,24.689293,16.921870,18.038637
3,0.126191,9.073919,47.598814
4,26.398543,50.755641,22.982564
5,30.104164,142.992524,33.507221
6,10.561219,13.544982,31.527813
7,20.878645,25.024393,27.506083
8,24.503416,123.955822,102.259554
9,10.548538,29.694862,0.640413


### Disyllablic vowel utterance 

In [29]:
def change_label_set_1(datapoint_df, col):
    datapoint_df.at['E', col]= 'ɛ:' 
    datapoint_df.at['O', col]= 'ɔ:'
    datapoint_df.at['9', col]= 'œ:'
    datapoint_df.at['@', col]= 'ə:'
    datapoint_df.at['o', col]= 'o:'
    datapoint_df.at['a', col]= 'a:'
    datapoint_df.at['i', col]= 'i:'
    datapoint_df.at['e', col]= 'e:'
    datapoint_df.at['u', col]= 'u:'
    datapoint_df.at['A', col]= 'ɑ:'
    datapoint_df.at['2', col]= 'ø:'
    datapoint_df.at['U', col]= 'ʊ:'

    return datapoint_df

def custome_reindex_type2(df):
    return df.reindex(["a:", "i:", "u:","e:",'ɛ:','ə:','œ:','o:','ɔ:', 'ɑ:','ø:','ʊ:'])

def set_datapoint_index(df, col):
    df_temp = df.copy()
    df_temp['Label_idx'] = df_temp[col]
    return df_temp.set_index('Label_idx')

In [93]:
exp_num = 140

In [94]:
path = '../experiment_2/result/eval_{}/formant/'.format(exp_num)


with open('../data/d_eval/syllable_name.txt') as f:
    label = np.array([word.strip()[0]+';'+word.strip()[1] for line in f for word in line.split(',')])
    
a1 = np.load(join(path, 'actual_F1.npy'))
a2 = np.load(join(path, 'actual_F2.npy'))
a3 = np.load(join(path, 'actual_F3.npy'))

e1 = np.load(join(path, 'estimated_F1.npy'))
e2 = np.load(join(path, 'estimated_F2.npy'))
e3 = np.load(join(path, 'estimated_F3.npy'))

e1 = numpy_fillna(e1)
e2 = numpy_fillna(e2)
e3 = numpy_fillna(e3)

f1_error = a1 - e1
f2_error = a2 - e2
f3_error = a3 - e3


f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))

f11_error = np.absolute(np.mean(f1_error[:,0,:], axis=1))
f21_error = np.absolute(np.mean(f1_error[:,1,:], axis=1))
f12_error = np.absolute(np.mean(f2_error[:,0,:], axis=1))
f22_error = np.absolute(np.mean(f2_error[:,1,:], axis=1))
f13_error = np.absolute(np.mean(f3_error[:,0,:], axis=1))
f23_error = np.absolute(np.mean(f3_error[:,1,:], axis=1))
    
    
res_df = pd.DataFrame({ 'Pho':label, '1F1 ERROR': f11_error,'1F2 ERROR':f12_error, '1F3 ERROR':f13_error,
              '2F1 ERROR': f21_error,'2F2 ERROR':f22_error, '2F3 ERROR':f23_error})

res_df['Pho1'], res_df['Pho2'] = res_df['Pho'].str.split(';', 1).str

In [95]:
res_df.head()

,Pho,1F1 ERROR,1F2 ERROR,1F3 ERROR,2F1 ERROR,2F2 ERROR,2F3 ERROR,Pho1,Pho2
0,a;i,3.721138,26.641260,49.638811,11.478654,20.129181,126.263750,a,i
1,a;u,4.298982,29.281510,50.904471,12.877228,47.955715,4.422733,a,u
2,a;e,4.302551,29.187249,51.037416,5.494803,3.468164,50.653916,a,e
3,a;E,3.740658,27.104133,50.545571,8.729425,28.549945,6.101305,a,E
4,a;o,4.254108,29.252248,51.086151,9.383158,34.026984,43.658125,a,o


In [96]:
res_1 = res_df.groupby(['Pho1']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

change_label_set_1(res_1, col='Pho1')
res_1 = set_datapoint_index(res_1, col='Pho1')
custome_reindex_type2(res_1).drop(['Pho1'], axis=1)

NameError: name 'change_label_set_1' is not defined

In [36]:
res_2 = res_df.groupby(['Pho2']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

change_label_set_1(res_2, col='Pho2')
res_2 = set_datapoint_index(res_2, col='Pho2')
custome_reindex_type2(res_2).drop(['Pho2'], axis=1)

,1F1 ERROR,1F2 ERROR,1F3 ERROR
Label_idx,,,
a:,13.635082,36.612971,40.965285
i:,11.146992,35.928388,24.943819
u:,12.421793,28.806141,43.113271
e:,12.694796,41.434180,44.453905
ɛ:,12.909823,36.603886,43.193302
ə:,11.916951,38.846014,43.410871
œ:,11.623216,37.032053,42.062268
o:,13.156882,31.715534,43.788306
ɔ:,12.709002,34.377758,42.457960


# Predict

In [16]:
def change_label_set_2(datapoint_df, col):
    datapoint_df.at['E', col]= 'ɛ:' 
    datapoint_df.at['O', col]= 'ɔ:'
    datapoint_df.at['7', col]= 'ɤ:'
    datapoint_df.at['M', col]= 'ɯ:'
    datapoint_df.at['o', col]= 'o:'
    datapoint_df.at['a', col]= 'a:'
    datapoint_df.at['i', col]= 'i:'
    datapoint_df.at['e', col]= 'e:'
    datapoint_df.at['u', col]= 'u:'
    return datapoint_df

def custome_reindex_type3(df):
    return df.reindex(["a:", "i:", "u:","e:",'ɛ:','ɯ:','ɤ:','o:','ɔ:'])

def set_datapoint_index(df, col):
    df_temp = df.copy()
    df_temp['Label_idx'] = df_temp[col]
    return df_temp.set_index('Label_idx')

In [9]:
EXP = ['145_1', '144_1','149','140_8', '143_1', '125','126','127_1','128']

In [22]:
exp_num = '129'

In [25]:
path = '../experiment_2/result/predict_{}/formant/'.format(exp_num)


with open('../data/d_records/selected/syllable_name.txt') as f:
    label = np.array([word.strip()[0]+';'+word.strip()[1] for line in f for word in line.split(',')])
    
a1 = np.load(join(path, 'actual_F1.npy'))
a2 = np.load(join(path, 'actual_F2.npy'))
a3 = np.load(join(path, 'actual_F3.npy'))

e1 = np.load(join(path, 'estimated_F1.npy'))
e2 = np.load(join(path, 'estimated_F2.npy'))
e3 = np.load(join(path, 'estimated_F3.npy'))

a1 = numpy_fillna(a1)
a2 = numpy_fillna(a2)
a3 = numpy_fillna(a3)

e1 = numpy_fillna(e1)
e2 = numpy_fillna(e2)
e3 = numpy_fillna(e3)

f1_error = np.absolute(a1[:,:] - e1[:,1:])/(a1[:,:]+1e-6)
f2_error = np.absolute(a2[:,:] - e2[:,1:])/(a2[:,:]+1e-6)
f3_error = np.absolute(a3[:,:] - e3[:,1:])/(a3[:,:]+1e-6)

f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))

f11_error = np.mean(f1_error[:,0,:], axis=1).astype('float')*100
f21_error = np.mean(f1_error[:,1,:], axis=1).astype('float')*100
f12_error = np.mean(f2_error[:,0,:], axis=1).astype('float')*100
f22_error = np.mean(f2_error[:,1,:], axis=1).astype('float')*100
f13_error = np.mean(f3_error[:,0,:], axis=1).astype('float')*100
f23_error = np.mean(f3_error[:,1,:], axis=1).astype('float')*100

    
res_df = pd.DataFrame({ 'Pho':label, '1F1 ERROR': f11_error,'1F2 ERROR':f12_error, '1F3 ERROR':f13_error,
              '2F1 ERROR': f21_error,'2F2 ERROR':f22_error, '2F3 ERROR':f23_error})

res_df['Pho1'], res_df['Pho2'] = res_df['Pho'].str.split(';', 1).str

In [26]:
res_df.describe()

,1F1 ERROR,1F2 ERROR,1F3 ERROR,2F1 ERROR,2F2 ERROR,2F3 ERROR
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,15.925777,15.906225,12.162396,14.905097,15.329667,13.184880
std,20.124404,14.146347,5.124059,13.820914,17.258163,5.162831
min,3.287185,1.060722,1.250273,2.913594,1.845970,2.874897
25%,8.380562,8.949313,7.781180,8.420925,8.310398,9.694920
50%,11.601480,12.426859,11.617330,12.526009,12.847388,13.255473
75%,19.278450,17.656064,15.590348,17.792097,16.281728,16.122742
max,181.586126,83.691404,31.464082,101.852607,146.685084,27.674255


In [27]:
res_1 = res_df.groupby(['Pho1']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

change_label_set_2(res_1, col='Pho1')
res_1 = set_datapoint_index(res_1, col='Pho1')
custome_reindex_type3(res_1).drop(['Pho1'], axis=1)

,1F1 ERROR,1F2 ERROR,1F3 ERROR
Label_idx,,,
a:,13.835049,11.620853,12.551416
i:,12.087812,13.765749,12.242500
u:,23.917266,44.157410,13.662704
e:,17.890259,12.159546,13.596946
ɛ:,18.265672,15.225981,13.797275
ɯ:,29.896487,14.181524,9.663066
ɤ:,8.713075,10.657127,12.100562
o:,7.864577,12.721451,6.557182
ɔ:,10.861793,8.666385,15.289916


In [28]:
res_2 = res_df.groupby(['Pho2']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

change_label_set_2(res_2, col='Pho2')
res_2 = set_datapoint_index(res_2, col='Pho2')
custome_reindex_type3(res_2).drop(['Pho2'], axis=1)

,1F1 ERROR,1F2 ERROR,1F3 ERROR
Label_idx,,,
a:,20.108681,18.506526,13.588098
i:,15.547602,12.953013,9.131595
u:,31.758579,24.299647,13.131558
e:,11.999352,13.099762,12.363108
ɛ:,13.400931,11.699733,11.805241
ɯ:,11.821002,12.606335,12.901697
ɤ:,14.585161,11.136883,12.422101
o:,13.371930,17.582579,12.644088
ɔ:,10.738750,21.271547,11.474082


# Compare Model (Predict)

### Disyllablic vowel utterance 

In [5]:
EXP = ['145_1', '144_1','149','140_8', '143_1', '125','126','127_1','128']

In [30]:
res = []
for exp_num in EXP:

    path = '../experiment_2/result/predict_{}/formant/'.format(exp_num)

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))
    a3 = np.load(join(path, 'actual_F3.npy'))
    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))
    e3 = np.load(join(path, 'estimated_F3.npy'))
    
    a1 = numpy_fillna(a1)
    a2 = numpy_fillna(a2)
    a3 = numpy_fillna(a3)
    
    e1 = numpy_fillna(e1)
    e2 = numpy_fillna(e2)
    e3 = numpy_fillna(e3)
    
#     print(a1[0])
#     print(e1[0,1:])
#     print(a1 - e1[:,:a1.shape[1]])
    

    f1_error = np.absolute(a1 - e1[:,1:])/(a1 + 1e-6)
    f2_error = np.absolute(a2 - e2[:,1:])/(a2+ 1e-6)
    f3_error = np.absolute(a3 - e3[:,1:])/(a3+ 1e-6)

    
    f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
    f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
    f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))
    
    f11_error = np.mean(f1_error[:,0,:])*100
    f21_error = np.mean(f1_error[:,1,:])*100
    f12_error = np.mean(f2_error[:,0,:])*100
    f22_error = np.mean(f2_error[:,1,:])*100
    f13_error = np.mean(f3_error[:,0,:])*100
    f23_error = np.mean(f3_error[:,1,:])*100

    res.append([exp_num, f11_error, f12_error, f13_error, f21_error, f22_error, f23_error])

pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR', '2F1 ERROR','2F2 ERROR', '2F3 ERROR'])

#     f11_error = np.mean(f1_error)*100
#     f12_error = np.mean(f2_error)*100
#     f13_error = np.mean(f3_error)*100
#     res.append([exp_num, f11_error, f12_error, f13_error])

# pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR'])

,MODEL,1F1 ERROR,1F2 ERROR,1F3 ERROR,2F1 ERROR,2F2 ERROR,2F3 ERROR
0,145_1,18.410161605944097,13.421312515466408,11.846692928020234,19.643876567364646,16.595749009829564,12.699890510782216
1,144_1,20.495661999537916,17.04654000471533,13.028616094167614,23.587618025979776,17.47508940736189,15.027931011181634
2,149,14.244943514956773,15.039131504950594,12.722631825208403,16.100456869485832,15.674847366380055,12.753664546445506
3,140_8,16.132702452677318,12.866797335626412,12.262064105025026,16.04382962330707,15.339438307068576,11.903937467019784
4,143_1,15.308163683032033,15.883562470412166,12.553595679847854,16.312663414276827,16.13246272770372,12.245996156978501
5,125,60.12363341868784,33.88666383554706,19.450202877343852,44.359231932608516,38.234148689667194,17.61587837117481
6,126,15.711122680039393,14.418446953580661,12.057433736383572,15.816971958503226,18.283016822535476,13.506579527505135
7,127_1,15.98463560047933,14.173283387947262,11.791060368243269,15.225755660692405,14.947075713242889,12.00336761919572
8,128,15.417329745865457,13.921887851124874,13.457957931607787,15.987682730477976,15.074476182869207,12.81982250677967


In [9]:
a1[:,:]

array([[905.2232450736384, 875.3850324344282, 880.9676400010229, ..., 0,
        0, 0],
       [567.1412858453965, 527.7297131803183, 484.40524522467933, ..., 0,
        0, 0],
       [1022.1660509226081, 975.0697177513188, 1004.2821026184486, ...,
        0, 0, 0],
       ...,
       [553.4840816614416, 557.5159977293788, 550.5065424593585, ..., 0,
        0, 0],
       [719.4078817457483, 723.5334914807275, 719.5770989615609, ..., 0,
        0, 0],
       [762.1240579861042, 758.9642437900978, 765.9878328523015, ..., 0,
        0, 0]], dtype=object)

In [89]:
res = []
for exp_num in EXP:

    path = '../experiment_2/result/predict_{}/formant/'.format(exp_num)

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))
    a3 = np.load(join(path, 'actual_F3.npy'))
    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))
    e3 = np.load(join(path, 'estimated_F3.npy'))
    
    a1 = numpy_fillna(a1)
    a2 = numpy_fillna(a2)
    a3 = numpy_fillna(a3)
    
    e1 = numpy_fillna(e1)
    e2 = numpy_fillna(e2)
    e3 = numpy_fillna(e3)
    

    f1_error = a1 - e1[:,1:]
    f2_error = a2 - e2[:,1:]
    f3_error = a3 - e3[:,1:]

    
#     f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
#     f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
#     f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))
    
#     f11_error = np.std(f1_error[:,0,:])
#     f21_error = np.std(f1_error[:,1,:])
#     f12_error = np.std(f2_error[:,0,:])
#     f22_error = np.std(f2_error[:,1,:])
#     f13_error = np.std(f3_error[:,0,:])
#     f23_error = np.std(f3_error[:,1,:])

    f11_error = np.std(f1_error)
    f12_error = np.std(f2_error)
    f13_error = np.std(f3_error)

#     res.append([exp_num, f11_error, f12_error, f13_error, f21_error, f22_error, f23_error])
    res.append([exp_num, f11_error, f12_error, f13_error])

# pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR', '2F1 ERROR','2F2 ERROR', '2F3 ERROR'])
pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR'])

,MODEL,1F1 ERROR,1F2 ERROR,1F3 ERROR
0,145_1,112.63166662902886,357.88906654595394,373.9314513757186
1,144_1,175.02366170492377,399.8194549537851,475.82294659649307
2,149,100.94761128382662,368.4575425437563,354.7726493069169
3,140_8,97.16321811567431,341.3264341242244,322.4220617840579
4,143_1,104.01637878459249,364.2676907230029,346.139583573839
5,125,421.64273005773174,660.1778239034861,886.801265905272
6,126,101.58323186850524,355.37709242428724,346.1072502128624
7,127_1,97.6679990108144,342.30268906870396,332.38360574356915
8,128,99.80314541007772,337.6671241982641,335.26480943493226


In [11]:
f1_error[:,0,:]

array([[-322.73438464504864, -72.48963865805672, 232.115639740554, ...,
        0, 0, 0],
       [-145.44469633072595, 494.328074980915, 357.7102471071097, ...,
        -677.4121722077449, 0, 0],
       [-245.2587294605247, 361.97275230528084, 201.89266691817295, ...,
        0, 0, 0],
       ...,
       [-885.3538404679518, -558.6593773510252, 108.55343484066577, ...,
        0, 0, 0],
       [-1526.9052579402062, 298.51467352435986, 147.00312709428795, ...,
        0, 0, 0],
       [-253.25402443291273, -69.50859609316683, 256.171453316303, ...,
        0, 0, 0]], dtype=object)

In [12]:
f1_error

array([[[-322.73438464504864, -72.48963865805672, 232.115639740554, ...,
         0, 0, 0],
        [-85.54668650649432, -113.10154910828726, -133.5090548288349,
         ..., 0, 0, 0]],

       [[-145.44469633072595, 494.328074980915, 357.7102471071097, ...,
         -677.4121722077449, 0, 0],
        [-102.79981643592157, -136.61865814199837, -139.16437188073837,
         ..., -150.02769338968773, 0, 0]],

       [[-245.2587294605247, 361.97275230528084, 201.89266691817295,
         ..., 0, 0, 0],
        [125.44044468098195, 142.96496350485768, 129.00131097663882,
         ..., 0, 0, 0]],

       ...,

       [[-885.3538404679518, -558.6593773510252, 108.55343484066577,
         ..., 0, 0, 0],
        [44.46079280247977, 45.95138342716439, 45.68595583977617, ...,
         0, 0, 0]],

       [[-1526.9052579402062, 298.51467352435986, 147.00312709428795,
         ..., 0, 0, 0],
        [115.86032479957515, 121.69313254646232, 111.41474624492042,
         ..., 0, 0, 0]],

       [[-253